# Dynamics of ODEs & Bifurcation Continuation

The [DifferentialEquations.jl](https://docs.sciml.ai/DiffEqDocs/stable/)
ecosystem is a collection of packages for solving differential equations.
`OrdinaryDiffEq` is the main package used for systems of ODEs, though
`DiffEqGPU` offers similar functionality for GPU acceleration.

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

using OrdinaryDiffEq

  Activating project at `~/Dev/siam-student-conf-2025/notebooks`


## Integrate-and-fire neuron model

<div style="background-color: white; text-align: center;">
  <img style="width: 320px;" src="https://neuronaldynamics.epfl.ch/online/x635.png">
</div>

*Image credit: https://neuronaldynamics.epfl.ch/online/Ch1.S3.html*

The integrate-and-fire model is based on a single simple ODE:

$$C\dot{V}(t) = g_{\rm Leak}(V(t)-V_{\rm Rest}) + I_{\rm Ext}(t),$$

where $C$ is the membrane capacitance, $g_{\rm Leak}$ is the membrane leak
conductance, $V_{\rm Rest}$ is the equilibrium potential of the leak channel,
and $I_{\rm Ext}(t)$ is a current applied to the membrane from the outside
world.

There are two extra parameters in the model:
1. $V_{\rm Th}$, a threshold voltage at which the neuron will "spike," or fire
   an action potential upon crossing upward.
2. $V_{\rm AP}$, the maximum voltage value of an action potential.

When an action potential is observed, the voltage $V$ should be immediately
reset to the equilibrium voltage value $V_{\rm Rest}$.

*Note: Formally speaking, the solutions to the integrate-and-fire neuron model
are distributions, where action potentials are instantaneous (Dirac delta
terms).*

### Derivative definition

In [9]:
# Define the differential equation.
# Note: The state V and derivatives dV are vectors (with only one component in
# this model). In general, they may have any number of components.
function integrate_and_fire!(
  dV::Vector{Float64}, # dV/dt. Will be mutated, hence `!` in the function name.
  V::Vector{Float64},  # Voltage.
  p::Vector{Float64},  # Parameter vector.
  t::Float64           # Time.
) # Since we're using an in-place solver, we mutate the existing dV instead of returning a new updated dV.
  # Extract parameters from p.
  C, g_leak, V_rest, _, _ = p
  
  # Compute the derivative.
  # Note: I_ext isn't defined yet -- we can call it here because Julia does
  # two-pass compilation. We'll define it later.
  dV[1] = (g_leak * (V_rest - V[1]) + I_ext(t)) / C;

  # We don't need to return anything for an in-place solver.
  # Also, note that we do NOT handle action potential events in this derivative
  # definition. Instead, we will use a callback injected into the numerical
  # solver -- DifferentialEquations.jl provides rich callback functionality.
  # We only need to define the derivatives for the model here.
end

integrate_and_fire! (generic function with 1 method)

### Callback definition (action potential handling)

https://docs.sciml.ai/DiffEqDocs/stable/features/callback_functions/

In [10]:
# We want to store the spike times. We reset this to nothing later, but
# I defined it here too for pedagogical clarity.
ap_times = Float64[]

# Define the action potential handler callback.
# This will be triggered when the voltage crosses the threshold from below to
# above.
function condition_spike(V, t, integrator)
  # Extract parameters from the integrator.
  _, _, V_rest, V_th, _ = integrator.p
  
  # Return the difference between voltage and threshold.
  # The callback triggers when this crosses zero from negative to positive.
  return V[1] - V_th
end

function affect_spike!(integrator)
  # Extract parameters from the integrator.
  _, _, V_rest, _, _ = integrator.p
  
  # Record that a spike occurred at the current time.
  append!(ap_times, integrator.t)
  
  # Reset voltage to resting potential.
  integrator.u[1] = V_rest
end

# Create the callback that will be passed to the solver.
#
# Because this is a *continuous* callback, `affect_spike!` will be called
# at exactly the time of the spike voltage threshold being crossed, no matter
# the integration time step size.
#
# The `affect_neg!` kwarg needs to be set to `nothing`; otherwise it defaults to
# whatever `affect!` is (in this case, `affect_spike!`). This would cause
# spikes to occur when the spike voltage threshold is crossed from above to
# below.
spike_callback = ContinuousCallback(condition_spike, affect_spike!; affect_neg! = nothing);

### External current

In [143]:
function I_ext(t::Float64)::Float64
  amplitude = 10.0
  period = 20.0

  # amplitude # Constant current.
  amplitude * ((sin(t * π / 2period) + 1.0) / 2.0)^3 # Oscillatory excitatory current.
  # amplitude * floor((t / period) % 2.0) # Periodic pulses.
end

I_ext (generic function with 1 method)

### Parameters & numerical solver config

In [155]:
# Define parameter vector.
p = [
  1.0,   # C
  0.01,  # g_leak
  -60.0, # V_rest
  -40.0, # V_th
  20.0   # V_ap
];
tspan = (0.0, 3e2); # Time span for integration.

### Initial condition

In [156]:
V = [p[3]]; # Start with voltage at resting voltage.
ap_times = Float64[]; # Reset spike times vector.

### Solve the system

In [157]:
# Set up the differential equation problem.
prob = ODEProblem(integrate_and_fire!, V, tspan, p)

# Compute the solution.
sol = solve(prob, Tsit5(), abstol=1e-8, reltol=1e-8, callback=spike_callback)

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 284-element Vector{Float64}:
   0.0
   0.02176125631892553
   0.23245492140641202
   0.6483645921415518
   1.1531436988007053
   1.7634220037532295
   2.457044195997477
   3.230118126336742
   4.066562294542153
   4.9574936071857225
   ⋮
 286.5001154775818
 288.6786596197479
 290.0797975256339
 291.74214450672076
 293.2411690756605
 294.8296644205109
 296.4284996679966
 298.14876907393494
 300.0
u: 284-element Vector{Vector{Float64}}:
 [-60.0]
 [-59.97273157834622]
 [-59.701720606501055]
 [-59.12828351288084]
 [-58.35987665943147]
 [-57.31713354824124]
 [-55.96881381844362]
 [-54.24446454402497]
 [-52.094059546571465]
 [-49.45297341164379]
 ⋮
 [-48.08371130413204]
 [-48.10662289630718]
 [-48.20953517086045]
 [-48.37284431242169]
 [-48.53647658660166]
 [-48.71446753902907]
 [-48.893011812389915]
 [-49.082410479823814]
 [-49.28266061789893]

### Plot the solution and external current

In [158]:
using GLMakie

In [159]:
# Create a figure with two panels.
fig = Figure(size=(800, 600))

# First panel for voltage.
ax1 = Axis(
  fig[1, 1], 
  xlabel="Time (ms)", 
  ylabel="Voltage (mV)",
  title="Membrane Potential"
)

# Plot the voltage trajectory.
lines!(ax1, sol.t, [u[1] for u in sol.u], linewidth=2, color=:blue)

# Add markers for action potentials.
if !isempty(ap_times)
  # Add vertical lines from threshold to action potential voltage.
  for t in ap_times
    lines!(
      ax1,
      [t, t],
      [p[4], p[5]],
      color=:blue,
      linewidth=2
    )
  end
  
  # Add scatter points at action potential peaks.
  scatter!(
    ax1,
    ap_times,
    fill(p[5], length(ap_times)), 
    color=:red,
    markersize=10
  )
end

# Add a horizontal line for threshold voltage.
hlines!(ax1, p[4], color=:black, linestyle=:dash, label="Threshold")

# Second panel for external current.
ax2 = Axis(
  fig[2, 1], 
  xlabel="Time (ms)", 
  ylabel="Current (nA)",
  title="External Current"
)

# Calculate current at each time point.
current_values = [I_ext(t) for t in sol.t]
lines!(ax2, sol.t, current_values, linewidth=2, color=:green)

# Add a horizontal line at zero current for reference.
hlines!(ax2, 0, color=:black, linestyle=:dash, label="Zero Current")

# Link the x-axes of both panels.
linkxaxes!(ax1, ax2)

# Add some padding to the layout.
fig[1:2, 1] = GridLayout(tellwidth=false)

# Display the figure.
fig